In [2]:
import numpy as np
import mxnet as mx
import pandas as pd

batch_size=5
vec_size=1

In [5]:
def get_mnist():
    mnist = fetch_mldata('MNIST original')
    np.random.seed(1234) # set seed for deterministic ordering
    p = np.random.permutation(mnist.data.shape[0])
    X = mnist.data[p]
    X = X.reshape((70000, 28, 28))

    X = np.asarray([cv2.resize(x, (64,64)) for x in X])

    X = X.astype(np.float32)/(255.0/2) - 1.0
    X = X.reshape((70000, 1, 64, 64))
    X = np.tile(X, (1, 3, 1, 1))
    X_train = X[:60000]
    X_test = X[60000:]

    return X_train, X_test

In [4]:
def setup_gen_and_disc():
    rand = mx.sym.Variable('rand')

    generator = mx.sym.FullyConnected(data=rand, name='generator_0', num_hidden=30)
#     generator.simple_bind(mx.cpu(), grad_req='null', generator_0_bias=np.array([14.7]))
    generator = mx.sym.Activation(data=generator, act_type='tanh')
#     generator = mx.sym.FullyConnected(data=generator, num_hidden=30)
#     generator = mx.sym.Activation(data=generator, act_type='tanh')
    generator = mx.sym.BlockGrad(generator)
    generator = mx.sym.FullyConnected(data=generator, name='generator_2', num_hidden=vec_size)
    gout = mx.sym.Activation(data=generator, act_type='sigmoid')
    
    data = mx.sym.Variable('data')
    label = mx.sym.Variable('label')

    disc = mx.sym.FullyConnected(data=data, name='discriminator_0', num_hidden=30)
#     disc.simple_bind(mx.cpu(), grad_req='null')
#     disc = mx.sym.Activation(data=disc, act_type='tanh')
#     disc = mx.sym.FullyConnected(data=disc, name='discriminator_1', num_hidden=30)
    disc = mx.sym.Activation(data=disc, act_type='tanh')
    disc = mx.sym.BlockGrad(disc)
    disc = mx.sym.FullyConnected(data=disc, name='discriminator_2', num_hidden=1)
    dloss = mx.sym.LogisticRegressionOutput(data=disc, label=label, name='dloss')
    
    return gout, dloss

symG, symD = setup_gen_and_disc()